In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pickle
import os

# Load dataset
DATASET_PATH = r"C:\Users\ADMIN\Documents\sign_language\sign_language_app\Gesture Image Pre-Processed Data"
IMG_SIZE = (224, 224)  # Increased from 128 to better utilize EfficientNet
BATCH_SIZE = 32

# Enhanced Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Save class labels
with open("classes.pkl", "wb") as f:
    pickle.dump(train_generator.class_indices, f)

# Load Pretrained EfficientNet with improved settings
base_model = EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling=None
)

# Gradual unfreezing strategy
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Build Enhanced Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Improved Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)]
)

# Callbacks for better training
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Train Model with more epochs
EPOCHS = 25
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

# Save Model
model.save("improved_sign_model.keras")

Found 44400 images belonging to 37 classes.
Found 11100 images belonging to 37 classes.
Epoch 1/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0641 - loss: 14.2346 - top_k_categorical_accuracy: 0.1621

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1928s 1s/step - accuracy: 0.0641 - loss: 14.2337 - top_k_categorical_accuracy: 0.1622 - val_accuracy: 0.4483 - val_loss: 9.6193 - val_top_k_categorical_accuracy: 0.7172 - learning_rate: 1.0000e-04
Epoch 2/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3474 - loss: 9.3476 - top_k_categorical_accuracy: 0.5939

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1809s 1s/step - accuracy: 0.3475 - loss: 9.3468 - top_k_categorical_accuracy: 0.5940 - val_accuracy: 0.7911 - val_loss: 5.7731 - val_top_k_categorical_accuracy: 0.9314 - learning_rate: 1.0000e-04
Epoch 3/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 887ms/step - accuracy: 0.6683 - loss: 5.5827 - top_k_categorical_accuracy: 0.8629

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1426s 1s/step - accuracy: 0.6684 - loss: 5.5822 - top_k_categorical_accuracy: 0.8630 - val_accuracy: 0.8987 - val_loss: 3.2511 - val_top_k_categorical_accuracy: 0.9755 - learning_rate: 1.0000e-04
Epoch 4/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8077 - loss: 3.1881 - top_k_categorical_accuracy: 0.9340

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 2321s 2s/step - accuracy: 0.8077 - loss: 3.1878 - top_k_categorical_accuracy: 0.9340 - val_accuracy: 0.9394 - val_loss: 1.7789 - val_top_k_categorical_accuracy: 0.9868 - learning_rate: 1.0000e-04
Epoch 5/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8747 - loss: 1.8129 - top_k_categorical_accuracy: 0.9627

...

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1263s 910ms/step - accuracy: 0.9634 - loss: 0.2880 - top_k_categorical_accuracy: 0.9921 - val_accuracy: 0.9823 - val_loss: 0.2151 - val_top_k_categorical_accuracy: 0.9973 - learning_rate: 1.0000e-04
Epoch 16/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1273s 917ms/step - accuracy: 0.9649 - loss: 0.2750 - top_k_categorical_accuracy: 0.9928 - val_accuracy: 0.9795 - val_loss: 0.2226 - val_top_k_categorical_accuracy: 0.9971 - learning_rate: 1.0000e-04
Epoch 17/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9669 - loss: 0.2708 - top_k_categorical_accuracy: 0.9930

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1694s 1s/step - accuracy: 0.9669 - loss: 0.2708 - top_k_categorical_accuracy: 0.9930 - val_accuracy: 0.9841 - val_loss: 0.2088 - val_top_k_categorical_accuracy: 0.9971 - learning_rate: 1.0000e-04
Epoch 18/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 864ms/step - accuracy: 0.9701 - loss: 0.2580 - top_k_categorical_accuracy: 0.9930

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1366s 984ms/step - accuracy: 0.9701 - loss: 0.2580 - top_k_categorical_accuracy: 0.9930 - val_accuracy: 0.9845 - val_loss: 0.1977 - val_top_k_categorical_accuracy: 0.9976 - learning_rate: 1.0000e-04
Epoch 19/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 0s 825ms/step - accuracy: 0.9681 - loss: 0.2578 - top_k_categorical_accuracy: 0.9939

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1318s 950ms/step - accuracy: 0.9681 - loss: 0.2578 - top_k_categorical_accuracy: 0.9939 - val_accuracy: 0.9880 - val_loss: 0.1855 - val_top_k_categorical_accuracy: 0.9974 - learning_rate: 1.0000e-04
Epoch 20/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1347s 970ms/step - accuracy: 0.9711 - loss: 0.2436 - top_k_categorical_accuracy: 0.9940 - val_accuracy: 0.9843 - val_loss: 0.1951 - val_top_k_categorical_accuracy: 0.9973 - learning_rate: 1.0000e-04
Epoch 21/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1467s 1s/step - accuracy: 0.9727 - loss: 0.2401 - top_k_categorical_accuracy: 0.9945 - val_accuracy: 0.9838 - val_loss: 0.1975 - val_top_k_categorical_accuracy: 0.9985 - learning_rate: 1.0000e-04
Epoch 22/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 2080s 1s/step - accuracy: 0.9749 - loss: 0.2306 - top_k_categorical_accuracy: 0.9947 - val_accuracy: 0.9806 - val_loss: 0.2008 - val_top_k_categorical_accuracy: 0.9982 - learning_rate: 1.0000e-04
Epoch 23/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1659s 1

1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1310s 944ms/step - accuracy: 0.9733 - loss: 0.2306 - top_k_categorical_accuracy: 0.9950 - val_accuracy: 0.9881 - val_loss: 0.1732 - val_top_k_categorical_accuracy: 0.9982 - learning_rate: 1.0000e-04
Epoch 25/25
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 1302s 938ms/step - accuracy: 0.9739 - loss: 0.2218 - top_k_categorical_accuracy: 0.9950 - val_accuracy: 0.9836 - val_loss: 0.1915 - val_top_k_categorical_accuracy: 0.9982 - learning_rate: 1.0000e-04
